## Dependencies

In [1]:
import glob
import html
import warnings
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, SpatialDropout1D, Concatenate, Add, Average
from googleqa_utilityscript import *
from transformers import BertConfig, BertTokenizer, TFBertModel


SEED = 0
seed_everything(SEED)
warnings.filterwarnings("ignore")

## Load data

In [2]:
MODULE_BASE_PATH = '/kaggle/input/bert-base-uncased-huggingface/bert-base-uncased'
MODULE_PATH = MODULE_BASE_PATH + '/bert-base-uncased-tf_model.h5'
VOCAB_PATH = MODULE_BASE_PATH + '/bert-base-uncased-vocab.txt'
model_path_list = glob.glob('/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/' + '*last_epoch.h5')
model_path_list += glob.glob('/kaggle/input/149-googleq-a-train-3fold-bert-base-unc-qa3-gpfold/' + '*last_epoch.h5')
model_path_list += glob.glob('/kaggle/input/150-googleq-a-train-3fold-bert-base-unc-qa3-adamw/' + '*last_epoch.h5')
model_path_list.sort()
print('Models to predict:')
print(*model_path_list, sep = "\n")

test = pd.read_csv('/kaggle/input/google-quest-challenge/test.csv')

print('Test samples: %s' % len(test))
display(test.head())

Models to predict:
/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/model_fold_1_last_epoch.h5
/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/model_fold_2_last_epoch.h5
/kaggle/input/148-googleq-a-train-3fold-bert-base-unc-qa3/model_fold_3_last_epoch.h5
/kaggle/input/149-googleq-a-train-3fold-bert-base-unc-qa3-gpfold/model_fold_1_last_epoch.h5
/kaggle/input/149-googleq-a-train-3fold-bert-base-unc-qa3-gpfold/model_fold_2_last_epoch.h5
/kaggle/input/149-googleq-a-train-3fold-bert-base-unc-qa3-gpfold/model_fold_3_last_epoch.h5
/kaggle/input/150-googleq-a-train-3fold-bert-base-unc-qa3-adamw/model_fold_1_last_epoch.h5
/kaggle/input/150-googleq-a-train-3fold-bert-base-unc-qa3-adamw/model_fold_2_last_epoch.h5
/kaggle/input/150-googleq-a-train-3fold-bert-base-unc-qa3-adamw/model_fold_3_last_epoch.h5
Test samples: 476


,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,host
0,39,Will leaving corpses lying around upset my pri...,I see questions/information online about how t...,Dylan,https://gaming.stackexchange.com/users/64471,There is no consequence for leaving corpses an...,Nelson868,https://gaming.stackexchange.com/users/97324,http://gaming.stackexchange.com/questions/1979...,CULTURE,gaming.stackexchange.com
1,46,Url link to feature image in the portfolio,I am new to Wordpress. i have issue with Featu...,Anu,https://wordpress.stackexchange.com/users/72927,I think it is possible with custom fields.\n\n...,Irina,https://wordpress.stackexchange.com/users/27233,http://wordpress.stackexchange.com/questions/1...,TECHNOLOGY,wordpress.stackexchange.com
2,70,"Is accuracy, recoil or bullet spread affected ...","To experiment I started a bot game, toggled in...",Konsta,https://gaming.stackexchange.com/users/37545,You do not have armour in the screenshots. Thi...,Damon Smithies,https://gaming.stackexchange.com/users/70641,http://gaming.stackexchange.com/questions/2154...,CULTURE,gaming.stackexchange.com
3,132,Suddenly got an I/O error from my external HDD,I have used my Raspberry Pi as a torrent-serve...,robbannn,https://raspberrypi.stackexchange.com/users/17341,Your Western Digital hard drive is disappearin...,HeatfanJohn,https://raspberrypi.stackexchange.com/users/1311,http://raspberrypi.stackexchange.com/questions...,TECHNOLOGY,raspberrypi.stackexchange.com
4,200,Passenger Name - Flight Booking Passenger only...,I have bought Delhi-London return flights for ...,Amit,https://travel.stackexchange.com/users/29089,I called two persons who work for Saudia (tick...,Nean Der Thal,https://travel.stackexchange.com/users/10051,http://travel.stackexchange.com/questions/4704...,CULTURE,travel.stackexchange.com


In [3]:
question_target_cols = ['question_asker_intent_understanding','question_body_critical', 'question_conversational', 
                        'question_expect_short_answer', 'question_fact_seeking', 'question_has_commonly_accepted_answer',
                        'question_interestingness_others', 'question_interestingness_self', 'question_multi_intent', 
                        'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice',
                        'question_type_compare', 'question_type_consequence', 'question_type_definition', 
                        'question_type_entity', 'question_type_instructions', 'question_type_procedure',
                        'question_type_reason_explanation', 'question_type_spelling', 'question_well_written']
answer_target_cols = ['answer_helpful', 'answer_level_of_information', 'answer_plausible', 'answer_relevance',
                      'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 
                      'answer_type_reason_explanation', 'answer_well_written']
target_cols = question_target_cols + answer_target_cols

In [4]:
### BERT auxiliar function
def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length, ignore_first_sep=True):
    """Segments: 0 for the first sequence, 1 for the second"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    segments = []
    current_segment_id = 0
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if ignore_first_sep:
                ignore_first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input_2_tail(ft_input1, ft_input2, tokenizer, max_sequence_length, 
                  max_len_ft1=254, max_len_ft2=255):

    ft1 = tokenizer.tokenize(ft_input1)
    ft2 = tokenizer.tokenize(ft_input2)
    
    ft1_len = len(ft1)
    ft2_len = len(ft2)

    if (ft1_len + ft2_len + 3) > max_sequence_length:
      
        if max_len_ft2 > ft2_len:
            ft2_new_len = ft2_len 
            ft1_new_len = max_len_ft1 + (max_len_ft2 - ft2_len)
        elif max_len_ft1 > ft1_len:
            ft2_new_len = max_len_ft2 + (max_len_ft1 - ft1_len)
            ft1_new_len = ft1_len
        else:
            ft2_new_len = max_len_ft2
            ft1_new_len = max_len_ft1
                        
        if ft1_new_len + ft2_new_len + 3 > max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d" 
                             % (max_sequence_length, (ft1_new_len + ft2_new_len + 3)))
        
        ft1 = ft1[-ft1_new_len:]
        ft2 = ft2[-ft2_new_len:]
        
    return ft1, ft2

def _convert_to_bert_inputs_2(feature1, feature2, tokenizer, max_sequence_length, ignore_first_sep=True):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    stoken = ["[CLS]"] + feature1 + ["[SEP]"] + feature2 + ["[SEP]"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length, ignore_first_sep)

    return [input_ids, input_masks, input_segments]

def compute_input_arays_2_tail(df, columns, tokenizer, max_sequence_length, 
                          max_len_ft1, max_len_ft2, ignore_first_sep=True):
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in df[columns].iterrows():
        ft1, ft2 = instance[columns[0]], instance[columns[1]]

        ft1, ft2 = _trim_input_2_tail(ft1, ft2, tokenizer, max_sequence_length, 
                                      max_len_ft1, max_len_ft2)

        ids, masks, segments = _convert_to_bert_inputs_2(ft1, ft2, tokenizer, max_sequence_length, 
                                                         ignore_first_sep)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
        
    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)]

## Pre-process data

In [5]:
text_features = ['question_title', 'question_body', 'answer']
text_features_question = ['question_title', 'question_body']
text_features_answer = ['question_title', 'answer']

test['question_title'] = test['question_title'].apply(html.unescape)

# Model parameters

In [6]:
N_CLASS = len(target_cols)
N_CLASS_QUESTION = len(question_target_cols)
N_CLASS_ANSWER = len(answer_target_cols)
MAX_SEQUENCE_LENGTH = 512
MAX_LENGTH_TITLE = 30
MAX_LENGTH_QUESTION = 479
MAX_LENGTH_ANSWER = 479

## Test set

In [7]:
tokenizer = BertTokenizer.from_pretrained(VOCAB_PATH, do_lower_case=True)

# Test features HEAD
X_test = compute_input_arays_2(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                               MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
        + compute_input_arays_2(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Test features TAIL
X_test_tail = compute_input_arays_2_tail(test, text_features_question, tokenizer, MAX_SEQUENCE_LENGTH, 
                                         MAX_LENGTH_TITLE, MAX_LENGTH_QUESTION, ignore_first_sep=False)\
              + compute_input_arays_2_tail(test, text_features_answer, tokenizer, MAX_SEQUENCE_LENGTH, 
                                           MAX_LENGTH_TITLE, MAX_LENGTH_ANSWER, ignore_first_sep=False)

# Model

In [8]:
bert_config = BertConfig()
bert_config.output_hidden_states=False

def model_fn():
    input_word_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_question')
    input_masks_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_question')
    segment_ids_question = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_question')

    input_word_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_word_ids_answer')
    input_masks_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='input_masks_answer')
    segment_ids_answer = Input((MAX_SEQUENCE_LENGTH,), dtype=tf.int32, name='segment_ids_answer')

    bert_model = TFBertModel.from_pretrained(MODULE_PATH, config=bert_config)
    sequence_output_question, pooled_output_question = bert_model([input_word_ids_question, input_masks_question, segment_ids_question])
    sequence_output_answer, pooled_output_answer = bert_model([input_word_ids_answer, input_masks_answer, segment_ids_answer])
    
    x_question = GlobalAveragePooling1D()(sequence_output_question)
    x_question = Dropout(0.2)(x_question)
    output_question = Dense(N_CLASS_QUESTION, kernel_initializer='glorot_uniform', activation="sigmoid", name="outputquestion")(x_question)
    
    x_answer = GlobalAveragePooling1D()(sequence_output_answer)
    x_answer = Dropout(0.2)(x_answer)
    output_answer = Dense(N_CLASS_ANSWER, kernel_initializer='glorot_uniform', activation="sigmoid", name="output_answer")(x_answer)
    
    output = Concatenate()([output_question, output_answer])

    model = Model(inputs=[input_word_ids_question, input_masks_question, segment_ids_question, 
                          input_word_ids_answer, input_masks_answer, segment_ids_answer], outputs=output)

    return model

# Make predictions

In [9]:
Y_test = np.zeros((len(test), N_CLASS))

for model_path in model_path_list:
    model = model_fn()
    model.load_weights(model_path)
    Y_test += ((model.predict(X_test) + model.predict(X_test_tail)) / 2) / len(model_path_list)

In [10]:
submission = pd.read_csv('/kaggle/input/google-quest-challenge/sample_submission.csv')
submission[target_cols] = Y_test


n=test['url'].apply(lambda x:(('ell.stackexchange.com' in x) or ('english.stackexchange.com' in x))).tolist()
spelling=[]
for x in n:
    if x:
        spelling.append(0.5)
    else:
        spelling.append(0.)
        
submission['question_type_spelling']=spelling


submission.to_csv("submission.csv", index=False)
display(submission.head())
display(submission.describe())

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.949999,0.610780,0.222834,0.571599,0.493308,0.521391,0.728716,0.672397,0.582151,...,0.917029,0.920068,0.593006,0.962875,0.975264,0.815495,0.015380,0.011479,0.855130,0.927102
1,46,0.893919,0.530911,0.006468,0.776676,0.761811,0.922438,0.561141,0.494183,0.084774,...,0.663554,0.936985,0.634834,0.971204,0.981277,0.883669,0.933534,0.130742,0.069117,0.904342
2,70,0.920333,0.723491,0.031989,0.768541,0.886902,0.934883,0.642677,0.503078,0.124973,...,0.906165,0.944066,0.628621,0.970217,0.976419,0.866279,0.021083,0.022881,0.883574,0.908507
3,132,0.871828,0.440029,0.007705,0.707061,0.731247,0.908623,0.536784,0.410354,0.091938,...,0.706398,0.942014,0.694459,0.957804,0.973911,0.903087,0.720036,0.131126,0.676435,0.906153
4,200,0.940225,0.482275,0.028049,0.882734,0.786127,0.929808,0.628657,0.629351,0.128907,...,0.632885,0.909990,0.652026,0.967291,0.966362,0.819428,0.272597,0.128748,0.531210,0.923810


,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
count,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,...,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000,476.000000
mean,5029.186975,0.907794,0.615113,0.038593,0.747964,0.800706,0.847890,0.596563,0.502492,0.229094,...,0.809295,0.934836,0.664097,0.964834,0.971698,0.868853,0.507433,0.124702,0.511727,0.916459
std,2812.670060,0.042613,0.153547,0.069758,0.101967,0.109740,0.125812,0.057795,0.093976,0.217598,...,0.094496,0.031293,0.050973,0.014921,0.016826,0.049578,0.329818,0.061553,0.297707,0.023609
min,39.000000,0.757732,0.286873,0.002585,0.227845,0.357348,0.202652,0.500998,0.333366,0.007033,...,0.543416,0.748292,0.503079,0.877151,0.828587,0.632649,0.005034,0.007970,0.021379,0.793345
25%,2572.000000,0.878852,0.484295,0.007760,0.692981,0.750964,0.820966,0.550193,0.432219,0.063297,...,0.730546,0.920806,0.631010,0.959242,0.966318,0.844609,0.167345,0.080351,0.251004,0.905314
50%,5093.000000,0.910248,0.599152,0.012151,0.749624,0.818538,0.892023,0.582007,0.475125,0.141956,...,0.808247,0.942689,0.661146,0.968066,0.976430,0.880813,0.581997,0.128358,0.515236,0.920756
75%,7482.000000,0.941359,0.750622,0.030807,0.813239,0.869436,0.923637,0.631034,0.552371,0.361787,...,0.901323,0.956240,0.692126,0.974662,0.981851,0.904737,0.816726,0.167054,0.793925,0.932871
max,9640.000000,0.986339,0.927347,0.550378,0.975256,0.984413,0.977956,0.760266,0.786302,0.941043,...,0.968276,0.982000,0.830749,0.987791,0.993575,0.956838,0.961832,0.314802,0.984681,0.961122
